# LAB 2 Andrea Mirenda s327644

The way to install any library using a linux terminal aproach is the following:

Import with predefined python environment

In [58]:
import pandas as pd
from dataclasses import dataclass
from collections import Counter
import random
import math
import numpy as np
from icecream import ic
from matplotlib import pyplot as plt
from itertools import accumulate
from itertools import combinations
from tqdm.auto import tqdm
import geopy.distance
from geopy.distance import geodesic
import networkx as nx
from collections import deque

In [59]:
cities = pd.read_csv('cities/italy.csv', header=None, names=['City', 'x', 'y'])

In [60]:
def find_closest(segments, city, visited):
    # Filtra i segmenti che contengono 'city' e hanno un'altra città non in 'visited'
    candidates = [
        (pair, distance) for pair, distance in segments
        if city in pair and (other_city := (pair - {city}).pop()) not in visited
    ]
    closest_segment = min(candidates, key=lambda x: x[1])
    closest_city = (closest_segment[0] - {city}).pop()
    return (int(city), closest_city)

def swap_mutation(solution, strength=0.5):
    while random.random()<strength:
        index = random.randint(1, len(solution) - 2) #not the first nor the last one
        index2=index #should be higher
        while index2<=index:
            index2 = random.randint(0, len(solution) - 1)
        # now we have the 2 indeces to use.
        selected_edge1 = solution[index]
        selected_edge2 = solution[index2]
        solution[index] = selected_edge2
        solution[index2] = selected_edge1
    #print("soluzione finale:", fitness(solution))
    return solution

def cyclic(solution):
    edges = []
    for node in range(len(solution)-2):
        edges.append({solution[node], solution[node+1]})
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return 1
    except:
        return 0

def num_cyclic(solution):
    edges = []
    for node in range(len(solution)-2):
        edges.append({solution[node], solution[node+1]})
    G = nx.Graph()
    G.add_edges_from(edges)
    cycles = nx.cycle_basis(G)
    return len(cycles)

def greedy_sol(city, segments):
    solution = []
    solution.append(city)
    visited = []
    visited.append(int(city))
    while len(visited)<len(cities):
        _, c1 = find_closest(segments, city, visited)
        solution.append(c1)
        visited.append(c1)
        city=c1
    solution.append(solution[0])
    
    return solution

def fitness(solution):
    tot_dist=0
    for node in range(len(solution)-1):
        tot_dist -= DIST_MATRIX[solution[node], solution[node+1]]
    # edges = []
    # for node in range(len(solution)-1):
    #     edges.append({solution[node], solution[node+1]})
    #print("gli archi sono ", edges)
    return (1-num_cyclic(solution),len(solution), tot_dist)
    #return (len(solution), tot_dist)

def parent_selection(population):
    candidates = sorted(np.random.choice(population, int(num_cities/5)), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


def inver_over_crossover(parent1, parent2):
    # Copia del primo genitore, su cui applicheremo le modifiche
    child = parent1[:]
    
    # Selezione di un punto di crossover casuale
    crossover_point = random.randint(1, len(parent1) - 2)
    
    # Segmento dal primo genitore (prima del punto di crossover)
    first_segment = parent1[:crossover_point]
    
    # Segmento dal secondo genitore (copia il collegamento per il resto)
    second_segment = []
    for city in parent2:
        if city not in first_segment:
            second_segment.append(city)
    
    # Combinazione dei segmenti per creare il figlio
    child = first_segment + second_segment
    
    return child

In [61]:
@dataclass
class Individual:
    genome: list
    fitness  = None

In [65]:
DIST_MATRIX = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.x, c1.y), (c2.x, c2.y)
    ).km
num_cities = len(cities)

POPULATION_SIZE = 20
segments = [
    ({c1, c2}, float(DIST_MATRIX[c1, c2])) for c1, c2 in combinations(range(len(cities)), 2)
]

population = []
for i in range(POPULATION_SIZE):
    sol = greedy_sol(random.randint(0, num_cities), segments)
    population.append(Individual(genome=sol))
#population = [Individual(greedy_sol(random.randint(0, num_cities), segments)) for _ in range(POPULATION_SIZE)]

for i in population:
    i.fitness = fitness(i.genome)
population.sort(key=lambda i: i.fitness, reverse=True)
print("best solution before EA: ", population[0].fitness)

OFFSPRING_SIZE = 10
MAX_GENERATIONS=10_000


for g in range(MAX_GENERATIONS):
    offspring = list()
    
    for _ in range (OFFSPRING_SIZE):
        if np.random.random()<.4:#mutation probability:
            p=parent_selection(population.copy())
            o=swap_mutation(p.genome.copy())
        else:
            i1 = parent_selection(population.copy())
            i2 = parent_selection(population.copy())
            o = inver_over_crossover(i1.genome.copy(), i2.genome.copy())
    offspring.append(Individual((o)))

for i in offspring:
    i.fitness = fitness(i.genome)
for i in population:
    print("fitness1", fitness(i.genome))
    print("fitness2", i.fitness)
    #i.fitness = fitness(i.genome)

population.extend(offspring)

population.sort(key=lambda i: i.fitness, reverse=True)
population = population[:POPULATION_SIZE]
solution = population[0].genome
print("best solution after EA: ", population[0].fitness)



best solution before EA:  (1, 47, np.float64(-4699.058487654932))
best solution after EA:  (1, 47, np.float64(-4699.058487654932))


Apply simulated annealing on the best solution:

In [63]:
#simulated annealing
temperatura_iniziale = 100
tasso_riscaldamento = 1.05
it=0

# def fitness(solution):
#     tot_dist=0
#     for node in range(len(solution)-1):
#         tot_dist -= DIST_MATRIX[solution[node], solution[node+1]]
    
#     return (len(solution), tot_dist)

#one out of five approach
miglioramenti_recenti = deque(maxlen=5) 
miglioramenti_richiesti = 1  

#stopping criteria:
miglioramenti_recenti_stop = deque(maxlen=100)
miglioramenti_recenti_stop.append(True)

# Initial solution: greedy one!
x_corrente = solution
costo_corrente = fitness(x_corrente)
best_cost = costo_corrente
best_sol = x_corrente
print("sol iniziale: ", fitness(solution))

temperatura = temperatura_iniziale
while it<1_000:
    it+=1
    # tweak the solution
    rn = random.random()
    first_time=True
    while rn > 0.5 or first_time:
        first_time=False
        x_nuovo = swap_mutation(x_corrente.copy())
        costo_nuovo = fitness(x_nuovo)
        rn = random.random()
    
    # variation of fitness by changing sign
    delta_costo = costo_nuovo[2]*(-1) - costo_corrente[2]*(-1) 
    #we are sure the solution after swap mutation is valid if the previous was it.
        
    if delta_costo < 0 or (random.random() < math.exp(-delta_costo / temperatura) and delta_costo!=0):
        x_corrente = x_nuovo
        costo_corrente = costo_nuovo
        miglioramenti_recenti.append(True)
        miglioramenti_recenti_stop.append(True)
        if costo_corrente[2]*(-1) < best_cost[2]*(-1):
            best_cost = costo_corrente
            best_sol = x_corrente
    else:
        miglioramenti_recenti.append(False)
        miglioramenti_recenti_stop.append(False)

    if miglioramenti_recenti.count(True) > miglioramenti_richiesti:
        temperatura *= tasso_riscaldamento  # more exploration
    if miglioramenti_recenti.count(True) < miglioramenti_richiesti:
        temperatura /= tasso_riscaldamento
    
    if miglioramenti_recenti_stop.count(True)==0: #stop condition
        break

fitness_new=fitness(best_sol)[2]*(-1)
print("solution after SA is: ", fitness_new)

sol iniziale:  (1, 47, np.float64(-4660.14905805592))
solution after SA is:  4660.14905805592
